## Q1:

In [1]:
!curl localhost:9200

{
  "name" : "97c57cd7a859",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "o1y98fDYRBWKOID7h703Mw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [3]:
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 7.4 MB/s eta 0:00:00


In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_Settings = {
    "settings": {
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}

        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_Settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 161.32it/s]


## Q3

In [17]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

resp = es_client.search(index=index_name, body=search_query)

for a in resp['hits']['hits']: 
    print (a["_score"])

84.050095


## Q4

In [21]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

resp = es_client.search(index=index_name, body=search_query)

nb = 0
for a in resp['hits']['hits']: 
    nb = nb + 1
    if nb==3:
        print (a["_source"]["question"])


How do I copy files from a different folder into docker container’s working directory?


In [23]:
def search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    resp = es_client.search(index=index_name, body=search_query)

    result_docs = []
    for a in resp['hits']['hits']: 
        result_docs.append(a["_source"])
    
    return result_docs



In [32]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()   


    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text'])+"\n\n"


    prompt = prompt_template.format(question=query, context=context)

    return prompt


In [33]:
query = "How do I execute a command in a running docker container?"

search_results = search(query)

len(build_prompt(query, search_results))

1492

In [34]:
!pip install tiktoken

  Using cached regex-2024.5.15-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 8.9 MB/s eta 0:00:00a 0:00:01
Using cached regex-2024.5.15-cp310-cp310-macosx_11_0_arm64.whl (278 kB)


In [47]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

prompt = build_prompt(query, search_results)

len(encoding.encode(prompt))



330

In [53]:
assert encoding.decode(encoding.encode(prompt)) == prompt


## Bonus question. 

I will use Groq to generate

In [57]:
from groq import Groq
from dotenv import dotenv_values

client = Groq(
    api_key=dotenv_values(".env")['GROQ_API_KEY']
    )


def llm(prompt):
    response = client.chat.completions.create(
        model='mixtral-8x7b-32768',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [59]:
response = rag(query)

In [60]:
len(encoding.encode(response))

134

## The cost is (we do it like if it were generated by OpenAI): 

In [62]:
(0.005 * len(encoding.encode(prompt)) + 0.015*len(encoding.encode(response)))/1000

0.00366